In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from copy import deepcopy
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Utility functions
def set_npseed(seed):
    np.random.seed(seed)

def set_torchseed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Data generation function
def data_gen_decision_tree(num_data=1000, dim=2, seed=0, w_list=None, b_list=None, 
                           vals=None, num_levels=3, threshold=1e-6, num_classes=4):        
    set_npseed(seed=seed)
    num_internal_nodes = 2**num_levels - 1
    num_leaf_nodes = 2**num_levels
    stats = np.zeros(num_internal_nodes+num_leaf_nodes)

    if vals is None:
        vals = np.arange(0, num_internal_nodes+num_leaf_nodes, 1, dtype=np.int32)
        vals[:num_internal_nodes] = -99
        
        # Assign classes to leaf nodes
        leaf_indices = np.arange(num_internal_nodes, num_internal_nodes+num_leaf_nodes)
        class_assignments = np.tile(np.arange(num_classes), (num_leaf_nodes // num_classes) + 1)[:num_leaf_nodes]
        np.random.shuffle(class_assignments)
        vals[leaf_indices] = class_assignments

    if w_list is None:
        w_list = np.random.randn(num_internal_nodes, dim)
        w_list /= np.linalg.norm(w_list, axis=1)[:, None]
        b_list = np.random.uniform(-0.5, 0.5, size=(num_internal_nodes,))

    oversample_factor = 2.0
    num_data_initial = int(num_data * oversample_factor)
    
    data_x = np.random.uniform(-1, 1, size=(num_data_initial, dim))
    relevant_stats = data_x @ w_list.T + b_list
    curr_index = np.zeros(shape=(num_data_initial), dtype=int)
    
    for level in range(num_levels):
        decision_variable = np.choose(curr_index, relevant_stats.T)
        curr_index = (curr_index+1)*2 - (1-(decision_variable > 0))
        
    bound_dist = np.min(np.abs(relevant_stats), axis=1)
    labels = vals[curr_index]
    
    data_x_pruned = data_x[bound_dist>threshold]
    labels_pruned = labels[bound_dist>threshold]
    
    samples_per_class = num_data // num_classes
    
    balanced_data = []
    balanced_labels = []
    
    for cls in range(num_classes):
        cls_mask = labels_pruned == cls
        cls_data = data_x_pruned[cls_mask]
        
        if len(cls_data) >= samples_per_class:
            indices = np.random.choice(len(cls_data), samples_per_class, replace=False)
            balanced_data.append(cls_data[indices])
            balanced_labels.append(np.full(samples_per_class, cls))
        else:
            indices = np.random.choice(len(cls_data), samples_per_class, replace=True)
            balanced_data.append(cls_data[indices])
            balanced_labels.append(np.full(samples_per_class, cls))
    
    data_x_balanced = np.vstack(balanced_data)
    labels_balanced = np.concatenate(balanced_labels)
    
    shuffle_idx = np.random.permutation(len(data_x_balanced))
    data_x_balanced = data_x_balanced[shuffle_idx]
    labels_balanced = labels_balanced[shuffle_idx]
    
    relevant_stats = np.sign(data_x_balanced @ w_list.T + b_list)
    nodes_active = np.zeros((len(data_x_balanced), num_internal_nodes+num_leaf_nodes), dtype=np.int32)
    for node in range(num_internal_nodes+num_leaf_nodes):
        if node==0:
            stats[node]=len(relevant_stats)
            nodes_active[:,0]=1
            continue
        parent = (node-1)//2
        nodes_active[:,node]=nodes_active[:,parent]
        right_child = node-(parent*2)-1
        if right_child==1:
            nodes_active[:,node] *= relevant_stats[:,parent]>0
        if right_child==0:
            nodes_active[:,node] *= relevant_stats[:,parent]<0
        stats = nodes_active.sum(axis=0)
    
    return ((data_x_balanced, labels_balanced), (w_list, b_list, vals), stats)

# DLGN Model (Multiclass)
class DLGN_FC(nn.Module):
    def __init__(self, input_dim=None, output_dim=None, num_hidden_nodes=[], 
                 beta=30, dlgn_mode='dlgn_sf', mode='pwc', num_classes=4):		
        super(DLGN_FC, self).__init__()
        self.num_hidden_layers = len(num_hidden_nodes)
        self.beta = beta
        self.dlgn_mode = dlgn_mode
        self.mode = mode
        self.num_classes = num_classes
        self.num_nodes = [input_dim] + num_hidden_nodes + [num_classes]
        self.gating_layers = nn.ModuleList()
        self.value_layers = nn.ModuleList()

        for i in range(self.num_hidden_layers+1):
            if i != self.num_hidden_layers:
                if self.dlgn_mode == 'dlgn_sf':
                    temp = nn.Linear(self.num_nodes[0], self.num_nodes[i+1], bias=False)
                else:
                    temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
                self.gating_layers.append(temp)
            temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
            self.value_layers.append(temp)
    
    def set_parameters_with_mask(self, to_copy, parameter_masks):
        for (name, copy_param) in to_copy.named_parameters():
            copy_param = copy_param.clone().detach()
            orig_param = self.state_dict()[name]
            if name in parameter_masks:
                param_mask = parameter_masks[name] > 0
                orig_param[param_mask] = copy_param[param_mask]
            else:
                orig_param = copy_param.data.detach()

    def return_gating_functions(self):
        effective_weights = []
        for i in range(self.num_hidden_layers):
            curr_weight = self.gating_layers[i].weight.detach().clone()
            if self.dlgn_mode == 'dlgn_sf':
                effective_weights.append(curr_weight)
            else:
                if i == 0:
                    effective_weights.append(curr_weight)
                else:
                    effective_weights.append(torch.matmul(curr_weight, effective_weights[-1]))
        return effective_weights

    def forward(self, x):
        gate_scores = [x]

        for el in self.parameters():
            if el.is_cuda:
                device = torch.device('cuda:0')
            else:
                device = torch.device('cpu')
        
        if self.mode == 'pwc':
            values = [torch.ones(x.shape).to(device)]
        else:
            values = [x.to(device)]

        for i in range(self.num_hidden_layers):
            if self.dlgn_mode == 'dlgn_sf':
                gate_scores.append((x @ self.gating_layers[i].weight.T))
            else:
                gate_scores.append(self.gating_layers[i].to(device)(gate_scores[-1].to(device)))
            curr_gate_on_off = torch.sigmoid(self.beta * gate_scores[-1])
            values.append(self.value_layers[i](values[-1]) * curr_gate_on_off)
        
        values.append(self.value_layers[self.num_hidden_layers](values[-1]))
        return values, gate_scores

# Training function (Multiclass)
def train_dlgn(DLGN_obj, train_data_curr, vali_data_curr, test_data_curr,
               train_labels_curr, test_labels_curr, vali_labels_curr, 
               num_epoch=1, parameter_mask=None, seed=42, lr=0.001, 
               no_of_batches=10, saved_epochs=None, x_epoch=1000):
    
    if parameter_mask is None:
        parameter_mask = {name: torch.ones_like(param) 
                         for name, param in DLGN_obj.named_parameters()}
    
    if saved_epochs is None:
        saved_epochs = list(range(0, num_epoch, 100)) + [num_epoch-1]
    
    set_torchseed(seed)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    DLGN_obj.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(DLGN_obj.parameters(), lr=lr)

    train_data_torch = torch.Tensor(train_data_curr)
    vali_data_torch = torch.Tensor(vali_data_curr)
    test_data_torch = torch.Tensor(test_data_curr)

    train_labels_torch = torch.tensor(train_labels_curr, dtype=torch.int64)
    test_labels_torch = torch.tensor(test_labels_curr, dtype=torch.int64)
    vali_labels_torch = torch.tensor(vali_labels_curr, dtype=torch.int64)

    num_batches = no_of_batches
    batch_size = len(train_data_curr) // num_batches
    losses = []
    DLGN_obj_store = []
    best_vali_error = len(vali_labels_curr)
    debug_models = []
    train_losses = []

    for epoch in tqdm(range(saved_epochs[-1]+1)):
        if epoch in saved_epochs:
            DLGN_obj_copy = deepcopy(DLGN_obj)
            DLGN_obj_copy.to(torch.device('cpu'))
            DLGN_obj_store.append(DLGN_obj_copy)
            
            train_outputs_values, train_outputs_gate_scores = DLGN_obj(
                torch.Tensor(train_data_curr).to(device))
            train_preds = train_outputs_values[-1]
            
            targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
            train_loss = criterion(train_preds, targets)

            train_losses.append(train_loss)
            if epoch % 1500 == 0 and epoch!=0:
                print(f"Epoch {epoch}, Train Loss: {train_loss.item():.6f}")
            if train_loss < 5e-6:
                break
            if np.isnan(train_loss.detach().cpu().numpy()):
                break
        
        running_loss = 0.0
        for batch_start in range(0, len(train_data_curr), batch_size):
            if (batch_start + batch_size) > len(train_data_curr):
                break
            
            optimizer.zero_grad()
            inputs = train_data_torch[batch_start:batch_start+batch_size]
            targets = train_labels_torch[batch_start:batch_start+batch_size].reshape(batch_size)
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            values, gate_scores = DLGN_obj(inputs)
            outputs = values[-1]
            loss = criterion(outputs, targets)
            loss.backward()
            
            for name, param in DLGN_obj.named_parameters():
                parameter_mask[name] = parameter_mask[name].to(device)
                param.grad *= parameter_mask[name]   
                if "gat" in name and epoch > x_epoch:
                    param.grad *= 0.
            
            optimizer.step()
            running_loss += loss.item()

        # Validation
        train_outputs_values, train_outputs_gate_scores = DLGN_obj(
            torch.Tensor(train_data_curr).to(device))
        train_preds = train_outputs_values[-1]
        targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
        train_loss = criterion(train_preds, targets)

        losses.append(train_loss.cpu().detach().clone().numpy())
        
        inputs = vali_data_torch.to(device)
        targets = vali_labels_torch.to(device)
        values, gate_scores = DLGN_obj(inputs)
        
        vali_preds = torch.argmax(values[-1], dim=1)
        vali_error = torch.sum(targets != vali_preds)
        
        if vali_error < best_vali_error:
            DLGN_obj_return = deepcopy(DLGN_obj)
            best_vali_error = vali_error
            
    DLGN_obj_return.to(torch.device('cpu'))
    return train_losses, DLGN_obj_return, DLGN_obj_store, losses, debug_models

# Baseline comparison functions
def train_baseline_models(train_data, train_labels, test_data, test_labels, num_classes):
    """Train and evaluate baseline ML models"""
    results = {}
    
    print("\n" + "="*60)
    print("Training Baseline Models")
    print("="*60)
    
    # Random Forest
    print("\nTraining Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(train_data, train_labels)
    rf_preds = rf.predict(test_data)
    rf_acc = accuracy_score(test_labels, rf_preds)
    results['Random Forest'] = {
        'accuracy': rf_acc,
        'predictions': rf_preds,
        'model': rf
    }
    print(f"Random Forest Test Accuracy: {rf_acc:.5f}")
    
    # SVM
    print("\nTraining SVM...")
    svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
    svm.fit(train_data, train_labels)
    svm_preds = svm.predict(test_data)
    svm_acc = accuracy_score(test_labels, svm_preds)
    results['SVM'] = {
        'accuracy': svm_acc,
        'predictions': svm_preds,
        'model': svm
    }
    print(f"SVM Test Accuracy: {svm_acc:.5f}")
    
    # CART (Decision Tree)
    print("\nTraining CART...")
    cart = DecisionTreeClassifier(max_depth=10, random_state=42)
    cart.fit(train_data, train_labels)
    cart_preds = cart.predict(test_data)
    cart_acc = accuracy_score(test_labels, cart_preds)
    results['CART'] = {
        'accuracy': cart_acc,
        'predictions': cart_preds,
        'model': cart
    }
    print(f"CART Test Accuracy: {cart_acc:.5f}")
    
    return results

def print_comparison_table(dlgn_acc, baseline_results, num_classes, test_labels):
    """Print comparison table of all models"""
    print("\n" + "="*60)
    print("Model Comparison")
    print("="*60)
    
    print(f"\n{'Model':<20} {'Test Accuracy':<15} {'Error Rate':<15}")
    print("-" * 50)
    print(f"{'DLGN':<20} {dlgn_acc:.5f} {' '*8} {1-dlgn_acc:.5f}")
    
    for model_name, results in baseline_results.items():
        acc = results['accuracy']
        print(f"{model_name:<20} {acc:.5f} {' '*8} {1-acc:.5f}")
    
    # Per-class accuracy comparison
    print(f"\n{'Model':<20}", end="")
    for i in range(num_classes):
        print(f"Class {i:<8}", end="")
    print()
    print("-" * (20 + num_classes * 15))
    
    # DLGN per-class
    print(f"{'DLGN':<20}", end="")
    for i in range(num_classes):
        class_mask = test_labels == i
        class_total = np.sum(class_mask)
        if class_total > 0:
            # This will be calculated outside and passed
            print(f"{0:.5f}{' '*3}", end="")
    print()
    
    # Baseline per-class
    for model_name, results in baseline_results.items():
        print(f"{model_name:<20}", end="")
        preds = results['predictions']
        for i in range(num_classes):
            class_mask = test_labels == i
            class_correct = np.sum((test_labels[class_mask] == preds[class_mask]))
            class_total = np.sum(class_mask)
            if class_total > 0:
                print(f"{class_correct/class_total:.5f}{' '*3}", end="")
        print()

# Main training loop
for num_data1 in [40000, 60000, 100000]:
    for num_classes1 in [3, 4, 5]:
        for input_dim1 in [30, 40, 50]:
            print("\n" + "="*60)
            print("DLGN Multiclass Classification with Baseline Comparison")
            print("="*60)
            
            # Configuration
            seed = 365
            num_levels = 4
            threshold = 0
            num_classes = num_classes1
            input_dim = input_dim1
            num_data = num_data1
            
            print(f"\nDataset Configuration:")
            print(f"  Input Dimension: {input_dim}")
            print(f"  Total Samples: {num_data}")
            print(f"  Number of Classes: {num_classes}")
            print(f"  Tree Levels: {num_levels}")
            
            # Generate data
            print("\nGenerating data...")
            ((data_x, labels), (w_list, b_list, vals), stats) = data_gen_decision_tree(
                dim=input_dim, 
                seed=seed, 
                num_levels=num_levels,
                num_data=num_data,
                num_classes=num_classes
            )
            
            print(f"\nLabel distribution:")
            for i in range(num_classes):
                print(f"  Class {i}: {sum(labels == i)}")
            
            # Split data
            num_train = num_data // 2
            num_vali = num_data // 4
            num_test = num_data // 4
            
            train_data = data_x[:num_train, :]
            train_data_labels = labels[:num_train]
            
            vali_data = data_x[num_train:num_train + num_vali, :]
            vali_data_labels = labels[num_train:num_train + num_vali]
            
            test_data = data_x[num_train + num_vali:, :]
            test_data_labels = labels[num_train + num_vali:]
            
            print(f"\nData split:")
            print(f"  Training: {len(train_data)}")
            print(f"  Validation: {len(vali_data)}")
            print(f"  Test: {len(test_data)}")
            
            # Model configuration
            dlgn_mode = 'dlgn'
            beta = 4
            lr = 0.002
            num_hidden_layers = 4
            num_hidden_nodes = [20, 20, 20, 20]
            no_of_batches = 10
            x_epoch = 1500
            saved_epochs = list(range(0, 1501, 100))
            
            print(f"\nDLGN Configuration:")
            print(f"  Mode: {dlgn_mode}")
            print(f"  Beta: {beta}")
            print(f"  Learning Rate: {lr}")
            print(f"  Hidden Layers: {num_hidden_layers}")
            print(f"  Hidden Nodes: {num_hidden_nodes}")
            
            # Initialize DLGN model
            set_torchseed(6675)
            DLGN_init = DLGN_FC(
                input_dim=input_dim, 
                output_dim=1,
                num_hidden_nodes=num_hidden_nodes, 
                beta=beta, 
                dlgn_mode=dlgn_mode,
                mode='pwc',
                num_classes=num_classes
            )
            
            # Setup parameter masks
            device = 'cpu'
            train_parameter_masks = dict()
            for name, parameter in DLGN_init.named_parameters():
                train_parameter_masks[name] = torch.ones_like(parameter).to(device)
            
            # Train DLGN model
            print("\n" + "="*60)
            print("Training DLGN...")
            print("="*60)
            
            set_torchseed(5000)
            train_losses, DLGN_obj_final, DLGN_obj_store, losses, debug_models = train_dlgn(
                train_data_curr=train_data,
                vali_data_curr=vali_data,
                test_data_curr=test_data,
                train_labels_curr=train_data_labels,
                vali_labels_curr=vali_data_labels,
                test_labels_curr=test_data_labels,
                DLGN_obj=deepcopy(DLGN_init),
                parameter_mask=train_parameter_masks,
                lr=lr,
                no_of_batches=no_of_batches,
                saved_epochs=saved_epochs,
                x_epoch=x_epoch
            )
            
            # Evaluate DLGN on test set
            test_outputs_values, test_outputs_gate_scores = DLGN_obj_final(torch.Tensor(test_data))
            test_logits = test_outputs_values[-1].detach().numpy()
            dlgn_preds = np.argmax(test_logits, axis=1)
            
            dlgn_acc = accuracy_score(test_data_labels, dlgn_preds)
            
            # Train baseline models
            baseline_results = train_baseline_models(
                train_data, train_data_labels, 
                test_data, test_data_labels, 
                num_classes
            )
            
            # Print comprehensive comparison
            print("\n" + "="*60)
            print("Comprehensive Model Comparison")
            print("="*60)
            
            print(f"\n{'Model':<20} {'Test Accuracy':<15} {'Error Rate':<15}")
            print("-" * 50)
            print(f"{'DLGN':<20} {dlgn_acc:.5f} {' '*8} {1-dlgn_acc:.5f}")
            
            for model_name, results in baseline_results.items():
                acc = results['accuracy']
                print(f"{model_name:<20} {acc:.5f} {' '*8} {1-acc:.5f}")
            
            # Per-class accuracy comparison
            print(f"\nPer-Class Accuracy:")
            print(f"{'Model':<20}", end="")
            for i in range(num_classes):
                print(f"Class {i:<10}", end="")
            print()
            print("-" * (20 + num_classes * 12))
            
            # DLGN per-class
            print(f"{'DLGN':<20}", end="")
            for i in range(num_classes):
                class_mask = test_data_labels == i
                class_correct = np.sum((test_data_labels[class_mask] == dlgn_preds[class_mask]))
                class_total = np.sum(class_mask)
                if class_total > 0:
                    print(f"{class_correct/class_total:.5f} {' '*3}", end="")
            print()
            
            # Baseline per-class
            for model_name, results in baseline_results.items():
                print(f"{model_name:<20}", end="")
                preds = results['predictions']
                for i in range(num_classes):
                    class_mask = test_data_labels == i
                    class_correct = np.sum((test_data_labels[class_mask] == preds[class_mask]))
                    class_total = np.sum(class_mask)
                    if class_total > 0:
                        print(f"{class_correct/class_total:.5f} {' '*3}", end="")
                print()
            
            print("\n" + "="*60)
            print("Experiment Complete!")
            print("="*60)


DLGN Multiclass Classification with Baseline Comparison

Dataset Configuration:
  Input Dimension: 30
  Total Samples: 40000
  Number of Classes: 3
  Tree Levels: 4

Generating data...

Label distribution:
  Class 0: 13333
  Class 1: 13333
  Class 2: 13333

Data split:
  Training: 20000
  Validation: 10000
  Test: 9999

DLGN Configuration:
  Mode: dlgn
  Beta: 4
  Learning Rate: 0.002
  Hidden Layers: 4
  Hidden Nodes: [20, 20, 20, 20]

Training DLGN...


100%|██████████| 1501/1501 [01:48<00:00, 13.89it/s]


Epoch 1500, Train Loss: 0.031172

Training Baseline Models

Training Random Forest...
Random Forest Test Accuracy: 0.65247

Training SVM...
SVM Test Accuracy: 0.77888

Training CART...
CART Test Accuracy: 0.53585

Comprehensive Model Comparison

Model                Test Accuracy   Error Rate     
--------------------------------------------------
DLGN                 0.90339          0.09661
Random Forest        0.65247          0.34753
SVM                  0.77888          0.22112
CART                 0.53585          0.46415

Per-Class Accuracy:
Model               Class 0         Class 1         Class 2         
--------------------------------------------------------
DLGN                0.92757    0.90186    0.88102    
Random Forest       0.75696    0.60777    0.59256    
SVM                 0.86770    0.73525    0.73339    
CART                0.58456    0.55274    0.47151    

Experiment Complete!

DLGN Multiclass Classification with Baseline Comparison

Dataset Configuration:


100%|██████████| 1501/1501 [01:50<00:00, 13.58it/s]


Epoch 1500, Train Loss: 0.076553

Training Baseline Models

Training Random Forest...
Random Forest Test Accuracy: 0.62476

Training SVM...
SVM Test Accuracy: 0.75058

Training CART...
CART Test Accuracy: 0.48575

Comprehensive Model Comparison

Model                Test Accuracy   Error Rate     
--------------------------------------------------
DLGN                 0.88719          0.11281
Random Forest        0.62476          0.37524
SVM                  0.75058          0.24942
CART                 0.48575          0.51425

Per-Class Accuracy:
Model               Class 0         Class 1         Class 2         
--------------------------------------------------------
DLGN                0.91077    0.85279    0.89815    
Random Forest       0.71680    0.58973    0.56684    
SVM                 0.83557    0.70827    0.70718    
CART                0.53924    0.44730    0.47045    

Experiment Complete!

DLGN Multiclass Classification with Baseline Comparison

Dataset Configuration:


...

100%|██████████| 1501/1501 [04:16<00:00,  5.84it/s]

Epoch 1500, Train Loss: 0.080675



Training Baseline Models

Training Random Forest...
Random Forest Test Accuracy: 0.52392

Training SVM...
SVM Test Accuracy: 0.74236

Training CART...
CART Test Accuracy: 0.36644

Comprehensive Model Comparison

Model                Test Accuracy   Error Rate     
--------------------------------------------------
DLGN                 0.94696          0.05304
Random Forest        0.52392          0.47608
SVM                  0.74236          0.25764
CART                 0.36644          0.63356

Per-Class Accuracy:
Model               Class 0         Class 1         Class 2         Class 3         Class 4         
--------------------------------------------------------------------------------
DLGN                0.95787    0.95311    0.93456    0.94144    0.94780    
Random Forest       0.43539    0.70725    0.53242    0.58355    0.35836    
SVM                 0.70907    0.83885    0.75761    0.76025    0.64425    
CART                0.33748    0.44858    0.37940    0.37686    0.28

100%|██████████| 1501/1501 [03:21<00:00,  7.45it/s]


Epoch 1500, Train Loss: 0.091058

Training Baseline Models

Training Random Forest...
Random Forest Test Accuracy: 0.50256

Training SVM...
SVM Test Accuracy: 0.70312

Training CART...
CART Test Accuracy: 0.34144

Comprehensive Model Comparison

Model                Test Accuracy   Error Rate     
--------------------------------------------------
DLGN                 0.93900          0.06100
Random Forest        0.50256          0.49744
SVM                  0.70312          0.29688
CART                 0.34144          0.65856

Per-Class Accuracy:
Model               Class 0         Class 1         Class 2         Class 3         Class 4         
--------------------------------------------------------------------------------
DLGN                0.91987    0.95611    0.93554    0.95121    0.93241    
Random Forest       0.45702    0.51583    0.41459    0.72621    0.40242    
SVM                 0.69458    0.72285    0.64001    0.81996    0.64016    
CART                0.28569    0.36